In [1]:
import iotbx.pdb
from scitbx.array_family import flex
from cctbx import crystal
import cctbx
import math
from iotbx.data_manager import DataManager
from itertools import chain
from collections import defaultdict
from scipy.spatial import KDTree 
import sys
import numpy as np
sys.path.append("../")
from qscore_utils import *
# dm = DataManager()
# dm.process_model_file("../data/1yjp.pdb")
# model = dm.get_model()

In [2]:
from qscore_combined import Qscore1_combined

In [3]:
# import numpy as np
# from scipy.spatial.distance import cdist
# def nd_to_1d_indices(indices, shape):
#     # Normalize indices to always use slice objects
#     normalized_indices = []
#     for dim, idx in enumerate(indices):
#         if idx is None:
#             normalized_indices.append(slice(0,shape[dim]))
#         else:
#             normalized_indices.append(idx)
    
#     # If any index is a slice, recursively call function for each value in slice
#     for dim, (i, s) in enumerate(zip(normalized_indices, shape)):
#         if isinstance(i, slice):
#             result_indices = []
#             start, stop, step = i.indices(s)
#             for j in range(start, stop, step):
#                 new_indices = list(normalized_indices)
#                 new_indices[dim] = j
#                 result_indices.extend(nd_to_1d_indices(new_indices, shape))
#             return result_indices
    
#     # If no slices, calculate single 1D index
#     index = 0
#     stride = 1
#     for i, dim in reversed(list(zip(normalized_indices, shape))):
#         index += i * stride
#         stride *= dim
#     return [index]

# def optimized_nd_to_1d_indices(i, shape):
#     # For fixed input of (None, i, None), we directly compute based on given structure
#     result_indices = []
    
#     # Pre-compute for 1st dimension which is always a slice
#     start1, stop1 = 0, shape[0]
    
#     # Pre-compute for 3rd dimension which is always a slice
#     start3, stop3 = 0, shape[2]
#     stride3 = 1
    
#     # Directly compute for 2nd dimension which is variable
#     stride2 = shape[2]
#     index2 = i * stride2 * shape[0]
    
#     for val1 in range(start1, stop1):
#         for val3 in range(start3, stop3):
#             result_indices.append(val1 * stride2 + index2 + val3 * stride3)
            
#     return result_indices



# def find_neighbors(A, B, radius):
#     """
#     Find neighbors within a given radius for each point in set B from set A.

#     Parameters:
#     - A (np.array): Set of 3D points of shape (M, 3).
#     - B (np.array): Set of 3D points of shape (N, 3).
#     - radius (float): The query radius.

#     Returns:
#     - neighbors (list of lists): For each point in B, a list of its neighbors' indices in A.
#     - distances (list of lists): For each point in B, a list of distances to its neighbors in A.
#     """
    
#     # Calculate the distance matrix
#     d_matrix = cdist(B, A)
    
#     # Find neighbors and their distances for each point in B
#     neighbors = [list(np.where(row < radius)[0]) for row in d_matrix]
#     distances = [list(row[row < radius]) for row in d_matrix]
    
#     return neighbors, distances


# def query_ball_point(points, query_point, radius, return_length=True):
#     """
#     For a given query point, find all points within a given radius.

#     Parameters:
#     - points (np.array): Set of 3D points of shape (N, 3).
#     - query_point (np.array): The 3D query point of shape (1, 3).
#     - radius (float): The query radius.
#     - return_length (bool, optional): If True, return only the count of neighbors.

#     Returns:
#     - indices (list): If return_length=False, indices of points within the radius. Else, count of such points.
#     """
    
#     # Calculate the distance from all points to the query point
#     distances = cdist(points, [query_point]).flatten()
    
#     # Find indices of points within the radius
#     indices = list(np.where(distances < radius)[0])
    
#     if return_length:
#         return len(indices)
#     else:
#         return indices



In [4]:
# from multiprocessing import Pool, cpu_count
# def radial_shell_combined_mp(model,n_probes=64,n_probes_target=8,radii=None,rtol=0.9,num_processes=cpu_count(),selection=None):

#     # Create argument tuples for each chunk
#     args = [(i,model,n_probes,n_probes_target,radius_shell,rtol,selection) for i,radius_shell in enumerate(radii)]
    
#     # Create a pool of worker processes
#     if num_processes >1:
#         with Pool(num_processes) as p:
#             # Use the pool to run the trilinear_interpolation_worker function in parallel
#             results = p.map(radial_shell_worker_combined, args)
#     else:
#         results = []
#         for arg in args:
#             result = radial_shell_worker_ccombined(arg)
#             results.append(result)

    
#     # stack numpy
#     probe_xyz_all = [result[0] for result in results]
#     keep_mask_all = [result[2] for result in results]
    

#     n_shells = len(radii)
#     n_atoms = probe_xyz_all[0].shape[0]
#     out_shape = (n_shells,n_atoms,n_probes,3 )
#     out_size = np.prod(out_shape)
#     shell_size = np.prod(out_shape[1:])
#     out_probes = np.full((n_shells,n_atoms,n_probes_target,3),-1.0)
#     out_mask = np.full((n_shells,n_atoms,n_probes_target),False)
    
#     for i,p in enumerate(probe_xyz_all):
#         out_probes[i,:,:n_probes_target,:] =p[:,:n_probes_target]

    
#     for i,k in enumerate(keep_mask_all):
#         start = i*shell_size
#         stop = start+shell_size
#         out_mask[i] = k[:,:n_probes_target]
#     probe_xyz_np = out_probes
#     keep_mask_np = out_mask
    
#     # stack flex
#     probe_xyz_all = [result[1] for result in results]
#     keep_mask_all = [result[3] for result in results]
    
    
#     n_shells = len(radii)
#     n_atoms = probe_xyz_all[0].focus()[0]
#     out_shape = (n_shells,n_atoms,n_probes_target,3 )
#     out_size = math.prod(out_shape)
#     shell_size = math.prod(out_shape[1:])
#     out_probes = flex.double(out_size,-1.0)
#     out_mask = flex.bool(n_atoms*n_shells*n_probes_target,False)
#     for i,p in enumerate(probe_xyz_all):
#         start = i*shell_size
#         stop = start+shell_size
#         out_probes = out_probes.set_selected(flex.uint32_range(start,stop),p.as_1d())
#     out_probes.reshape(flex.grid(*out_shape))

    
#     for i,k in enumerate(keep_mask_all):
#         start = i*(n_atoms*n_probes_target)
#         stop = start+(n_atoms*n_probes_target)
#         out_mask = out_mask.set_selected(flex.uint32_range(start,stop),k.as_1d())
#     out_mask.reshape(flex.grid(n_shells,n_atoms,n_probes_target))
    
#     probe_xyz_flex = out_probes
#     keep_mask_flex = out_mask
    
#     return probe_xyz_np, probe_xyz_flex, keep_mask_np, keep_mask_flex

In [5]:
dm = DataManager()
dm.process_model_file("../data/6cvm.cif")
dm.process_real_map_file("../data/emd_7770.map")
mmm = dm.get_map_model_manager()
assert flex.sum(mmm.map_manager().density_at_sites_cart(mmm.model().get_sites_cart()))>0,(
    "Negative or zero density around atoms. Map/model likely not aligned")
model = mmm.model()
mm = mmm.map_manager()
M = mm.map_data()

In [6]:
from qscore_combined import radial_shell_combined_mp

In [7]:

model = mmm.model()
mm = mmm.map_manager()
M = mm.map_data()
radii = np.linspace(0,2,16)
n_probes = 8
selection = model.selection("chain A and resseq 1:10")
num_processes = 1
rtol = 0.9
(probe_xyz, 
 probe_xyz_cctbx, 
 keep_mask, 
 keep_mask_cctbx) = radial_shell_combined_mp(model,
                                              n_probes=n_probes,
                                              radii=radii,
                                              rtol = rtol,
                                              selection=selection,
                                              num_processes=num_processes)

  0%|          | 0/16 [00:00<?, ?it/s]

In [10]:
Qscore1_combined(mmm,num_processes=1,selection = model.selection("chain A and resseq 1:10"))

  0%|          | 0/20 [00:00<?, ?it/s]

NameError: name 'probe_xyz_flex' is not defined

In [35]:
mask_clash=True
voxel_size=1.0
n_probes=8
min_probes=1
radii=np.arange(0.1,2.1,0.1)
rtol=0.9
ignore_min_probes=False
selection = model.selection("chain A and resseq 1:10")
num_processes = 6



probe_xyz_np, probe_xyz_flex, keep_mask_np, keep_mask_flex = radial_shell_combined_mp(model,radii=radii,n_probes=16,n_probes_target=8,selection=selection)

In [36]:
assert np.all(np.isclose(probe_xyz_np, probe_xyz_flex.as_numpy_array()))
assert np.all(keep_mask_np==keep_mask_flex.as_numpy_array())

In [38]:
probe_xyz = probe_xyz_np
keep_mask = keep_mask_np
probe_xyz_cctbx = probe_xyz_flex
keep_mask_cctbx = keep_mask_flex

In [39]:
do_np = True
do_flex = True
do_test = False

# to test must do both
if do_test:
    do_np = True
    do_flex = True
# PROCEED



# flatten numpy arrays
if do_np:
    n_shells,n_atoms,n_probes,_ = probe_xyz.shape
    probe_xyz_flat = probe_xyz.reshape((n_atoms*n_shells*n_probes,3))
    keep_mask_flat = keep_mask.reshape(-1) # (n_shells*n_atoms*n_probes,)
    

# init cctbx arrays

if do_flex:
    n_shells,n_atoms,n_probes,_ = probe_xyz_flex.focus()
    probe_xyz_cctbx = flex.double(probe_xyz)
    keep_mask_cctbx = flex.bool(keep_mask)




# APPLY MASK BEFORE INTERPOLATION

# numpy
if do_np:
    masked_probe_xyz_flat = probe_xyz_flat[keep_mask_flat]

# cctbx
if do_flex:
    keep_mask_cctbx_fullflat = []

    for val in keep_mask_cctbx:
        for _ in range(3):  # since A has an additional dimension of size 3
            keep_mask_cctbx_fullflat.append(val)

    mask = flex.bool(keep_mask_cctbx_fullflat)
    #indices = flex.int([i for i in range(1, keep_mask_cctbx.size() + 1) for _ in range(3)])
    sel = probe_xyz_cctbx.select(mask)
    #sel_indices = indices.select(mask)
    masked_probe_xyz_flat_cctbx = flex.vec3_double(sel)




In [40]:
# INTERPOLATE

# numpy
if do_np:
    M = mm.map_data()
    volume = M.as_numpy_array()
    voxel_size = np.array(mm.pixel_sizes())
    masked_density = trilinear_interpolation(volume, masked_probe_xyz_flat, voxel_size=voxel_size)

# cctbx
if do_flex:
    masked_density_cctbx = mm.density_at_sites_cart(masked_probe_xyz_flat_cctbx)

    # test equivalent
    if do_test:
        assert np.all(np.isclose(masked_density,masked_density_cctbx.as_numpy_array()))


# reshape interpolated values to (n_shells,n_atoms, n_probes)

# numpy
if do_np:
    d_vals = np.zeros((n_shells, n_atoms, n_probes))
    d_vals[keep_mask] = masked_density

# cctbx
if do_flex:
    keep_mask_cctbx.reshape(flex.grid(n_shells*n_atoms*n_probes))
    d_vals_cctbx = flex.double(keep_mask_cctbx.size(),0.0)
    d_vals_cctbx = d_vals_cctbx.set_selected(keep_mask_cctbx,masked_density_cctbx)
    d_vals_cctbx.reshape(flex.grid(n_shells,n_atoms,n_probes))

    if do_test:
        # test
        assert np.all(np.isclose(d_vals,d_vals_cctbx.as_numpy_array()))


# reshape to (M,N*L) for rowwise correlation

# numpy
if do_np:
    d_vals_2d = d_vals.transpose(1,0,2).reshape(d_vals.shape[1], -1)

# cctbx
if do_flex:
    def custom_reshape_indices(flex_array):
        N,M,L = flex_array.focus()
        result = flex.double(flex.grid(M, N * L))

        for i in range(N):
            for j in range(M):
                for k in range(L):
                    # Calculate the original flat index
                    old_index = i * M * L + j * L + k
                    # Calculate the new flat index after transpose and reshape
                    new_index = j * N * L + i * L + k
                    result[new_index] = flex_array[old_index]

        return result

    d_vals_2d_cctbx = custom_reshape_indices(d_vals_cctbx)


    # test
    if do_test:
        assert np.all(np.isclose(d_vals_2d,d_vals_2d_cctbx.as_numpy_array()))


# create the reference data

# numpy
if do_np:
    M = mm.map_data().as_numpy_array()
    maxD = min(M.mean()+M.std()*10,M.max())
    minD = max(M.mean()-M.std()*1,M.min())
    A = maxD-minD
    B = minD
    u = 0
    sigma = 0.6
    x = np.array(radii)
    y = A * np.exp(-0.5*((x-u)/sigma)**2) + B 




#cctbx
if do_flex:
    M = mm.map_data()
    maxD_cctbx = min(flex.mean(M)+M.standard_deviation_of_the_sample()*10,flex.max(M))
    minD_cctbx = max(flex.mean(M)-M.standard_deviation_of_the_sample()*1,flex.min(M))
    A_cctbx = maxD_cctbx-minD_cctbx
    B_cctbx = minD_cctbx
    u = 0
    sigma = 0.6
    x = flex.double(radii)
    y_cctbx = A_cctbx * flex.exp(-0.5*((flex.double(x)-u)/sigma)**2) + B_cctbx




    # test
    if do_test:
        assert np.all(np.isclose(np.array(y_cctbx),y))


# Stack and reshape data for correlation calc

# numpy
if do_np:
    # stack the reference to shape (n_shells,n_atoms,n_probes)
    g_vals = np.repeat(y[:,None],n_probes,axis=1)
    g_vals = np.expand_dims(g_vals,1)
    g_vals = np.tile(g_vals,(n_atoms,1))


    # reshape
    g_vals_2d = g_vals.transpose(1,0,2).reshape(g_vals.shape[1], -1)
    d_vals_2d = d_vals.transpose(1,0,2).reshape(d_vals.shape[1], -1)
    mask_2d = keep_mask.transpose(1,0,2).reshape(keep_mask.shape[1], -1)



# cctbx
if do_flex:
    # 1. Repeat y for n_probes (equivalent to np.repeat)
    g_vals_cctbx = [[val] * n_probes for val in y_cctbx]

    # 2. Add a new dimension (equivalent to np.expand_dims)
    g_vals_expanded = [[item] for item in g_vals_cctbx]

    # 3. Tile for each atom (equivalent to np.tile)
    g_vals_tiled = []
    for item in g_vals_expanded:
        g_vals_tiled.append(item * n_atoms)


    g_vals_cctbx = flex.double(np.array(g_vals_tiled) )


    # test 
    if do_test:
        assert np.all(np.isclose(g_vals_cctbx.as_numpy_array(),g_vals))


# # CALCULATE Q

# # numpy
if do_np:
    q = rowwise_corrcoef(g_vals_2d,d_vals_2d,mask=mask_2d)



# cctbx
if do_flex:
    d_vals_cctbx = d_vals_cctbx.as_1d()
    g_vals_cctbx = g_vals_cctbx.as_1d()
    keep_mask_cctbx_double = keep_mask_cctbx.as_1d().as_double()
    q_cctbx = []
    for atomi in range(n_atoms):

        #inds = nd_to_1d_indices((None,atomi,None),(n_shells,n_atoms,n_probes))
        inds = optimized_nd_to_1d_indices(atomi,(n_shells,n_atoms,n_probes))
        inds = flex.uint32(inds)
        d_row = d_vals_cctbx.select(inds)
        
        if do_test:
            assert np.all(np.isclose(d_row.as_numpy_array(),d_vals_2d[atomi]))

        g_row = g_vals_cctbx.select(inds)
        if do_test:
            assert np.all(np.isclose(g_row.as_numpy_array(),g_vals_2d[atomi]))

        mask = keep_mask_cctbx.select(inds)
        if do_test:
            assert np.all(np.isclose(mask.as_numpy_array(),mask_2d[atomi]))

        d = d_row.select(mask)
        g = g_row.select(mask)
        qval = flex.linear_correlation(d,g).coefficient()
        q_cctbx.append(qval)
    

    q_cctbx = flex.double(q_cctbx)
    if do_test:
        assert np.all(np.isclose(q,np.array(q_cctbx)))

In [41]:
q

array([ 1.64840792e-17,  2.45751023e-17,  0.00000000e+00,  5.99714904e-18,
        0.00000000e+00,  5.81031901e-17,  3.71147537e-17, -7.09197339e-17,
        3.77665092e-17,  2.08951328e-17, -6.37055373e-17,  3.74357560e-17,
        5.81556119e-18,  0.00000000e+00, -9.36591433e-18,  1.23085050e-17,
        0.00000000e+00, -2.01572320e-17,  4.67672103e-17,  1.34623744e-17,
       -2.99346027e-17, -1.77864806e-17,  2.93908615e-17, -3.90477276e-17,
        0.00000000e+00, -2.41990440e-17,  0.00000000e+00,  2.71549486e-18,
        7.40005440e-17,  0.00000000e+00,  0.00000000e+00, -2.11092161e-17,
       -1.31652185e-17,  4.53198060e-17,  1.10948691e-17,  0.00000000e+00,
        1.04422268e-17,  0.00000000e+00,  3.56932413e-17,  2.18366163e-17,
        0.00000000e+00, -4.20066059e-17,  5.98342913e-17, -5.01339265e-17,
       -1.32897009e-17, -3.91604854e-17,  2.44047688e-17,  0.00000000e+00,
       -1.36006992e-17,  0.00000000e+00, -3.53883341e-17, -2.05801223e-17,
        2.32039736e-17, -